<a href="https://colab.research.google.com/github/sweetpand/Logo_detection_PyTorch/blob/master/Logo_recognition_networks_%7C_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and import all the necessary libraries

In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu100' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip3 install -q http://download.pytorch.org/whl/{accelerator}/torch-1.1.0-{platform}-linux_x86_64.whl 
!pip3 install torchvision



import torch
import urllib.request
import os
import zipfile
import shutil
from pathlib import Path
%matplotlib inline
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from PIL import Image

#Download the training, validation and test

In [0]:
!unzip -uq "/content/FlickrLogos-32_dataset_v2" -d "/content/dataset"

In [0]:
!wget http://www.multimedia-computing.de/flickrlogos/data/FlickrLogos-32_dataset_v2.zip

# Train the network function

In [0]:
def train_val(model, criterion, optimizer, num_epochs=20):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoc {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
            
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # Forward pass
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    # Compute loss
                    loss = criterion(outputs, labels)

                    # Compute gradients and update parameters if train
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size()[0]
                running_corrects += torch.sum(preds == labels).item()

            epoch_loss = running_loss / len(datasets[phase])
            epoch_acc = running_corrects / len(datasets[phase])
            if phase == 'train':
                train_losses.append(epoch_loss)
                print('##: {}', train_losses)
            else:
                val_losses.append(epoch_loss)
                print('##: {}', val_losses)
        
            print('{} Loss: {:.4f} Acc.: {:.2f} %'.format(
                phase.title(), epoch_loss, epoch_acc * 100))
        
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Accuracy: {:.2f} %'.format(best_acc * 100))
    model.load_state_dict(best_model_wts)
    
    return model

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load the data

In [0]:
DATA_DIR = Path("dataset/FlickrLogos-v2/classes/jpg")
SETS = ['train', 'val', 'test']

In [0]:
SOURCE_DIR = Path('dataset/FlickrLogos-v2')
DATA_DIR = Path('data')
SETS = ['train', 'val', 'test']

In [0]:
def load_datasets(url, dst_dir):
    zip_file = url.split(sep='/')[-1]
    if not dst_dir.is_dir():
        if not zip_file.is_file():
            urllib.request.urlretrieve(url, zip_file)        
        with zipfile.ZipFile(zip_file) as zip_ref:
            zip_ref.extractall()
        #os.unlink(zip_file)    # comment out if you want to delete zip file

#Prepare the data

In [0]:
def prepare_datasets(src_dir, dst_dir, keep_source=True):
    for dataset, paths in dataset_paths.items():
        num_files = 0
        for path in paths:
            num_files += 1
            src = src_dir / path
            dst = dst_dir / (path.replace('classes/jpg', dataset))
            dst.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src, dst)
        print(dataset, 'dataset:', str(num_files))
    if not keep_source: shutil.rmtree(src_dir)

In [0]:
def list_image_paths(txt_relpath):
    with open(txt_relpath) as f:
        image_paths = f.read().splitlines()
    return image_paths

In [0]:
train_logo_relpaths = list_image_paths(SOURCE_DIR / 'trainset.relpaths.txt')
val_logo_relpaths = list_image_paths(SOURCE_DIR / 'valset-logosonly.relpaths.txt')
val_nologo_relpaths = list_image_paths(SOURCE_DIR / 'valset-nologos.relpaths.txt')
test_relpaths = list_image_paths(SOURCE_DIR / 'testset.relpaths.txt')

In [0]:
train_relpaths = train_logo_relpaths + val_nologo_relpaths[:int(len(val_nologo_relpaths) / 2)]
val_relpaths = val_logo_relpaths + val_nologo_relpaths[int(len(val_nologo_relpaths) / 2):]

In [0]:
relpaths = [train_relpaths, val_relpaths, test_relpaths]
dataset_paths = dict(zip(SETS, relpaths))

In [0]:
prepare_datasets(SOURCE_DIR, DATA_DIR)

In [0]:
train_mean = np.array([0.485, 0.456, 0.406])
train_std = np.array([0.229, 0.224, 0.225])

##Make to change resize to 299X299 for INCEPTION_V3

In [0]:
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)), 
#     transforms.Resize((299, 299)), inception
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std)
])

In [0]:
datasets = {i : torchvision.datasets.ImageFolder(DATA_DIR / i, data_transforms) 
            for i in ['train', 'val', 'test']}

In [0]:
bz = 8
dataloaders = {i : DataLoader(datasets[i], batch_size=bz, shuffle=(i == 'train'), num_workers=0) 
               for i in ['train', 'val', 'test']}

In [0]:
def imshow(img):
    npimg = img.numpy().transpose((1, 2, 0))
    npimg = npimg * train_std + train_std    # denorm
    npimg = np.clip(npimg, 0, 1)
    plt.imshow(npimg)

In [0]:
imgs, labels = next(iter(dataloaders['train']))

In [0]:
img = torchvision.utils.make_grid(imgs[:4])
classes = datasets['train'].classes
print(', '.join(classes[i] for i in labels[:4]))
imshow(img)
print(classes)

###SqueezeNet

In [0]:
model_ft = torchvision.models.squeezenet1_0(pretrained=True)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)

###Inception V3

In [0]:
model_ft = torchvision.models.inception_v3(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 100)
model_ft.aux_logits=False # inception
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)

###ResNet50

In [0]:
model_ft = torchvision.models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 100)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)

###GoogleNet

In [0]:
model_ft = torchvision.models.googlenet(pretrained=True)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)

###VGG16

In [0]:
model_ft = torchvision.models.vgg16(pretrained=True)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)


---
# Train the network



In [0]:
train_losses, val_losses = [], []

%time model_ft = train_val(model_ft, criterion, optimizer_ft)
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend(frameon=False)
plt.show()

# Save the model

In [0]:
torch.save(model_ft, 'logo_NameOfTheNetwork.pth')